In [2]:
import requests 
import pandas as pd 
import numpy as np
from dateutil import parser 

## Accéder à l'identifiant d'un film : cas pour un film 

In [36]:
key_api='fd0f74b9c4e9f318e9cb00ca6fec7c3b' #clé de l'API 

In [37]:
film='The Fabelmans'
url=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={film}" 

#requête pour trouver les infos  
req = requests.get(url)
carte = req.json()

ind=[]
for film in range(len(carte['results'])):
    ind.append(carte['results'][film]['id'])
print(ind)  

[804095, 1201609]


## Généralisation pour plusieurs films 

#### fonction pour recuperer l'id des films 

In [38]:
def id_recup(titre):
    url_api=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={titre}" 
    req = requests.get(url_api)
    carte = req.json()

    ind=[]
    for film in range(len(carte['results'])):
        ind.append(carte['results'][film]['id'])
    return(ind)

#### fonction pour récuperer les infos du film à partir de l'id 

In [39]:
def df_avec_id(id):
    id_film= id
    url_new_api = f"https://api.themoviedb.org/3/movie/{id_film}?api_key={key_api}&language=en-US"
    req_new = requests.get(url_new_api)
    wb_new = req_new.json()
    
    
    #ajustement des données 
    if 'belongs_to_collection' in wb_new and wb_new['belongs_to_collection'] is not None:
        wb_new['belongs_to_collection'] = wb_new['belongs_to_collection']['name']

        
    # Ajustement des données pour la clé 'genres'
    if 'genres' in wb_new:
        wb_new['genres'] = ', '.join([x['name'] for x in wb_new['genres']])
    else:
        wb_new['genres'] = None

# Ajustement des données pour la clé 'production_companies'
    if 'production_companies' in wb_new:
        wb_new['production_companies'] = ', '.join([x['name'] for x in wb_new['production_companies']])
    else:
        wb_new['production_companies'] = None

# Ajustement des données pour la clé 'production_countries'
    if 'production_countries' in wb_new:
        wb_new['production_countries'] = ', '.join([x['name'] for x in wb_new['production_countries']])
    else:
        wb_new['production_countries'] = None

# Ajustement des données pour la clé 'spoken_languages'
    if 'spoken_languages' in wb_new:
        wb_new['spoken_languages'] = ', '.join([x['name'] for x in wb_new['spoken_languages']])
    else:
        wb_new['spoken_languages'] = None

    
    df=pd.DataFrame(wb_new, index=[0])
    
    return (df)

#### Création de la database 

In [40]:
def get_movie_info(movie_id_list):
    # Initialiser un DataFrame vide pour stocker les informations sur les films
    movie_df = pd.DataFrame()

    for movie_id in movie_id_list:
        # Utiliser la deuxième fonction pour obtenir les informations détaillées du film
        movie_info = df_avec_id(movie_id)

        # Vérifier si des informations ont été trouvées
        if movie_info is not None:
            # Ajouter les informations du film au DataFrame
            movie_df = pd.concat([movie_df, movie_info], ignore_index=True)

    return movie_df

#Exemple 
#movie_id_list = get_id("Inception") 
#result_df = get_movie_info(movie_id_list)
#result_df.head()

In [41]:
def create_movie_list(movie_list):
    all_movies_df = pd.DataFrame()

    for movie_name in movie_list:
        movie_id_list = id_recup(movie_name)
        if movie_id_list:
            movie_info_df = get_movie_info(movie_id_list)
            # Ajouter les informations du film au DataFrame global
            all_movies_df = pd.concat([all_movies_df, movie_info_df], ignore_index=True)

    return all_movies_df

# Exemple 
#movie_list = ["Inception", "The Dark Knight", "Interstellar"]
#result_df = create_movie_list_dataframe(movie_list)
#result_df.head()

#### recuperer les titres des films sous forme de liste 

In [42]:
df_movies= pd.read_csv("df_film_avec_critiques.csv")
liste_des_films = df_movies['Titre original'].tolist()
#print(liste_des_films)
len(liste_des_films)

4194

In [43]:
moitie = len(liste_des_films) // 2

# Séparation de la liste en deux parties
liste_partie_1 = liste_des_films[:moitie]
liste_partie_2 = liste_des_films[moitie:]

moit = len(liste_partie_1)//2
half = len(liste_partie_2)//2

quart= liste_partie_1[:moit]
deux_quart= liste_partie_1[moit:]
trois_quart= liste_partie_2[:half]
quatre_quart= liste_partie_2[half:]


# Affichage des deux listes résultantes
print("Partie 1:", len(quart))
print("Partie 2:", len(deux_quart))
print("Partie 3:", len(trois_quart))
print("Partie 4:", len(quatre_quart))

Partie 1: 1048
Partie 2: 1049
Partie 3: 1048
Partie 4: 1049


#### DATAFRAME 

In [44]:
%%time 
df_movie_info1=create_movie_list(quart)

CPU times: user 3min 37s, sys: 17.1 s, total: 3min 54s
Wall time: 21min 6s


In [45]:
%%time
df_movie_info2=create_movie_list(deux_quart)

CPU times: user 3min 54s, sys: 20.2 s, total: 4min 14s
Wall time: 21min 41s


In [46]:
%%time
df_movie_info3=create_movie_list(trois_quart)

CPU times: user 3min 52s, sys: 19.9 s, total: 4min 12s
Wall time: 20min 36s


In [46]:
%%time
df_movie_info4=create_movie_list(quatre_quart)

CPU times: user 3min 52s, sys: 19.9 s, total: 4min 12s
Wall time: 20min 36s


In [51]:
#passage de chaque dataframe intermediaire en csv pour les sauvgarder

dataframes = [df_movie_info1, df_movie_info2, df_movie_info3, df_movie_info4]

for i, df in enumerate(dataframes, start=1):
    nom_fichier_csv = f'df_movie_info{i}.csv'
    df.to_csv(nom_fichier_csv, index=False)

In [54]:
# Rassembler les DataFrames en un seul DataFrame
df_tmdb = pd.concat(dataframes, ignore_index=True)

# Afficher le DataFrame final
df_tmdb.head()

# Sauvegarder le DataFrame tmdb en CSV
df_tmdb.to_csv('df_tmdb.csv', index=False)
